<a href="https://colab.research.google.com/github/Sanali209/pythontools/blob/main/Image_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [40]:
#enabling full width display of jupyter notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#installing pre-requisites
! pip install imagededup

In [46]:
from imagededup.methods import CNN
from tqdm import tqdm
import numpy as np
import time
from typing import Dict
import ipywidgets as widgets
from IPython.display import display,clear_output
import hashlib
import os.path
import pickle
import logging
from typing import Any
import json
from enum import Enum
import shutil


## globall settings

In [4]:
EncodingHash_Path ='D:/temp/EncodingHash'
save_dub_data_path = 'D:/temp/dub_data.json'

# defain background code

In [5]:
def get_files(path, exts=None, file_ignore_masck=""):
    if exts is None:
        exts = ["*"]
    import os
    import fnmatch
    matches = []
    for root, dirnames, filenames in os.walk(path):
        for file in filenames:
            if fnmatch.fnmatch(file, file_ignore_masck):
                continue
            for ext in exts:
                # no case match
                if fnmatch.fnmatch(file, ext):
                    matches.append(os.path.join(root, file))
    return matches


class SegmentedFilesHash(dict[str, Any]):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self._filepath = args[0] if len(args) > 0 else kwargs.get("filepath", None)
        if self._filepath is None:
            raise Exception("filepath is not set")
        else:
            if self._filepath.endswith("\\") or self._filepath.endswith("/"):
                self._filepath = self._filepath[:-1]
            if not os.path.exists(self._filepath):
                os.makedirs(self._filepath)
            self._LoadKeys()

    def _SaveKeys(self):
        path = os.path.join(self._filepath, "keys.pk")
        with open(path, "wb") as f:
            pickle.dump(list(self.keys()), f)

    def _LoadKeys(self):
        path = os.path.join(self._filepath, "keys.pk")
        if not os.path.exists(path):
            return
        with open(path, "rb") as f:
            try:
                keys = pickle.load(f)
            except Exception as e:
                logging.log(logging.ERROR, e, exc_info=True, stack_info=True)
                return
        self.update({key: None for key in keys})

    def _SaveItem(self, key, value):
        # konvert key to md5
        md5key = hashlib.md5(key.encode()).hexdigest()
        itempath = os.path.join(self._filepath, md5key)
        with open(itempath, "wb") as f:
            pickle.dump(value, f)

    def _DeleteItem(self, key):
        itempath = os.path.join(self._filepath, key)
        if os.path.exists(itempath):
            os.remove(itempath)

    def _LoadItem(self, key):
        # konvert key to md5
        md5key = hashlib.md5(key.encode()).hexdigest()
        itempath = os.path.join(self._filepath, md5key)
        if not os.path.exists(itempath):
            return False, None
        with open(itempath, "rb") as f:
            try:
                item = pickle.load(f)
            except Exception as e:
                logging.log(logging.ERROR, e, exc_info=True, stack_info=True)
                return False, None
            return True, item

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self._SaveItem(key, value)
        self._SaveKeys()

    def __getitem__(self, key):
        if key not in self:
            raise KeyError(key)
        item = super().__getitem__(key)
        if item is None:
            loaded, item = self._LoadItem(key)
            if loaded:
                self[key] = item
            else:
                raise KeyError(key)
        return super().__getitem__(key)

    def __delitem__(self, key):
        super().__delitem__(key)
        self._DeleteItem(key)
        self._SaveKeys()

class DuplicateFindHelper:

    def __init__(self):
        self.image_paths = None
        self.AnnoyIndex = None
        self.futuresMap = None
        self.cnn_encoder = CNN(verbose=True)
        self.EncodingHash = SegmentedFilesHash(EncodingHash_Path)

    def RefreshIndeksDir(self, dirPath):
        imagePats = get_files(dirPath, ['*.jpg', '*.png', '*.jpeg'])
        self.RefreshIndex(imagePats)

    def RefreshIndex(self, ImagePats: list[str]):
        futuresMap = self.CreateCNNEncoding(ImagePats)
        keylist = list(futuresMap.keys())
        self.image_paths = keylist

       

    def FindTopSimilar(self, ImagePath: str, TopCount: int) -> list[str]:
        v = self.GetCNNEncoding(ImagePath)
        u = self.AnnoyIndex
        # u.load(config.image_features_vectors_ann)  # super fast, will just mmap the file
        index_list = u.get_nns_by_vector(v, TopCount)  # will find the 10 nearest neighbors

        listSimPats = []
        for i in index_list:
            listSimPats.append(self.image_paths[i])

        return listSimPats

    def CreateCNNEncoding(self, imagePaths: list) -> dict:

        if imagePaths is None:
            raise Exception("imagePaths is None")

        if len(imagePaths) == 0:
            raise Exception("imagePaths is empty")

        encodingMap = {}

        progress = tqdm(imagePaths)
        for path in progress:

            if path in self.EncodingHash:
                value = self.EncodingHash[path]
                encodingMap[path] = self.BinToNPArray(value)
            else:
                ThumbPath = path
                try:
                    future = self.cnn_encoder.encode_image(ThumbPath)
                except Exception as e:
                    print("wrong file: " + path)
                    print(e)
                    # assign tu future blank np.array shape(1,576)
                    future = [np.zeros((256, 1))]
                    continue
                self.EncodingHash[path] = self.NPArrayToBin(future[0])
                encodingMap[path] = future[0]
        return encodingMap

    def GetCNNEncoding(self, imagePath: str) -> np.array:
        if imagePath == None:
            raise Exception("imagePath is None")
        if len(imagePath) == 0:
            raise Exception("imagePath is empty")

        future = self.cnn_encoder.encode_image(imagePath)
        return future[0]

    def FaindCNNDubs(self, encodingMap: Dict[str, list], similarity: float = 0.85, score=True) -> dict:
        """use wiz CreateCNNEncoding
        Sample:
        listOfFiles = get_files(self.ProcesingPath, exts=['*.jpg', '*.png', '*.jpeg'])
        encodingmap = DuplicateFindHelper.CreateCNNEncoding(listOfFiles)
        dublicates = DuplicateFindHelper.FaindCNNDubs(encodingmap, similarity=0.85)
        """

        result = self.cnn_encoder.find_duplicates(image_dir=None, encoding_map=encodingMap,
                                                  min_similarity_threshold=similarity, scores=score)
        return result

    def FaindItemsAsFolder(self, directory, etalondirectory, similarity: float = 0.85) -> dict:

        etalonImages = get_files(etalondirectory, ['*.jpg', '*.jpeg', '*.png', '*.bmp'])
        gallimages = get_files(directory, ['*.jpg', '*.jpeg', '*.png', '*.bmp'])

        return None

    @staticmethod
    def NPArrayToArry(array: np.array) -> list:
        return array.tolist()

    def NPArrayToBin(self, array: np.array) -> bytes:
        return array.tobytes()

    def BinToNPArray(self, array: bytes) -> np.array:
        return np.frombuffer(array, dtype=np.float32)

    @staticmethod
    def ArryToNPArray(array: list) -> np.array:
        return np.array(array)

    @staticmethod
    def ClearEmptyDubsGroup(dubDictionary: dict[str, list]):
        """remove empty dubs group from dictionary"""
        for key in dubDictionary.copy():
            if len(dubDictionary[key]) == 0:
                del dubDictionary[key]

    @staticmethod
    def ClearFullCollidedDubsGroup(dubDictionary: dict[str, list]):
        currentTime = time.time()
        remKeys = set()

        listOfDirKeys = list(dubDictionary.keys())
        tqdmprogress = tqdm(range(len(listOfDirKeys)),total=len(listOfDirKeys), desc="ClearFullCollidedDubsGroup")
        for i in tqdmprogress:
            path1 = listOfDirKeys[i]


            if path1 in remKeys:
                continue

            for j in range(i + 1, len(listOfDirKeys)):

                path2 = listOfDirKeys[j]
                if path2 in remKeys:
                    continue
                items1 = {path1, *[item[0] for item in dubDictionary[path1]]}
                items2 = {path2, *[item[0] for item in dubDictionary[path2]]}
                # if sets equal
                if items1 == items2:
                    remKeys.add(path2)
                    break
                # compare if all items present in list2
                if items2.issubset(items1):
                    remKeys.add(path2)
                    break
                # compare if all items present in list1
                if items1.issubset(items2):
                    remKeys.add(path1)
                    break

        for key in remKeys:
            del dubDictionary[key]
        lastTime = time.time()
        print(f"removed {len(remKeys)} dubs")

        return len(remKeys), lastTime - currentTime


# dub faind

In [35]:

path = r'F:\rawimagedb\repository\safe repo'

listOfFiles = get_files(path, exts=['*.jpg', '*.png', '*.jpeg'])
dubfainder = DuplicateFindHelper()
encodingmap = dubfainder.CreateCNNEncoding(listOfFiles)
dublicates = dubfainder.FaindCNNDubs(encodingmap, similarity=0.9)
dubfainder.ClearEmptyDubsGroup(dublicates)
dubfainder.ClearFullCollidedDubsGroup(dublicates)

# def group fields enum
class GroupOptions(Enum):
    id = 0
    name = 1
    selected = 2
    images = 3
    processingOptions = 4
    optionArgs = 5


# convert data format for frendlier use
def convert(dublicates):
    dublist = []
    counter = 0
    for key, value in dublicates.items():
        baseimagepath = key
        id = counter
        relatedimages = value
        dubgroup = {'id': id, 'name':id, 'selected': False, 'relatedimages': [], 'groupoptions': 'none', 'optionargs': 'none'}
        dubgroup['relatedimages'].append(
            {'relatedimagepath': baseimagepath, 'selected': False, 'confidence': '1', 'id': id,'groupoptions': 'none', 'optionargs': 'none'})
        for relatedimagepath, confidence in relatedimages:
            counter += 1
            id = counter
            dubgroup['relatedimages'].append(
                {'relatedimagepath': relatedimagepath, 'selected': False, 'confidence': str(confidence), 'id': id,
                 'groupoptions': 'none', 'optionargs': 'none'})
        dublist.append(dubgroup)
        counter += 1
    return dublist

#convert file list for gui drawing
def convertfilelist(listOfFiles):
    filelist = []
    counter = 0
    for filepath in listOfFiles:
        filelist.append({'filepath': filepath, 'selected': False, 'id': counter})
        counter += 1
    return filelist

list_files = convertfilelist(listOfFiles)

listdublicates = convert(dublicates)

print(len(listdublicates))



2023-05-17 11:45:46,460: INFO Initialized: MobileNet v3 pretrained on ImageNet dataset sliced at GAP layer
2023-05-17 11:45:46,461: INFO Device set to cpu ..
100%|██████████| 12776/12776 [01:30<00:00, 140.79it/s]
2023-05-17 11:47:17,244: INFO Start: Calculating cosine similarities...
2023-05-17 11:47:17,245: INFO Large feature matrix thus calculating cosine similarities in chunks...
100%|██████████| 13/13 [00:06<00:00,  1.94it/s]
2023-05-17 11:47:24,665: INFO End: Calculating cosine similarities.
ClearFullCollidedDubsGroup: 100%|██████████| 259/259 [00:00<00:00, 32390.86it/s]

removed 124 dubs
135


## save dubsearch results

In [37]:
#save as json in utf-8
with open(save_dub_data_path, 'w', encoding='utf-8') as f:
    json.dump(listdublicates, f, ensure_ascii=False, indent=4)
    print('saved')

saved


## load dubsearch results

In [ ]:
#load as json in utf-8
with open(save_dub_data_path, 'r', encoding='utf-8') as f:
    listdublicates = json.load(f)

## Gui code

In [52]:

# don't now need this handler?
def set_selected_group(dublicates, val):
    id = val['owner'].id
    selected = val['new']

    for group in dublicates:
        if group['id'] == id:
            group['selected'] = selected

            break
        for relatedimage in group['relatedimages']:
            if relatedimage['id'] == id:
                relatedimage['selected'] = selected
                break

# Define a function to handle the dropdown value change event
def group_dropdown_changed(change):
    new_value = change['new']
    ovner = change['owner']
    group = ovner.group
    group['groupoptions'] = new_value

# Define a function to handle the group args value change event in text box
def group_args_changed(change):
    new_value = change['new']
    ovner = change['owner']
    group = ovner.group
    group['optionargs'] = new_value
    print(new_value)

# Define a function to handle the dropdown value change event
def image_dropdown_changed(change):
    new_value = change['new']
    ovner = change['owner']
    image = ovner.image
    image['groupoptions'] = new_value


# Define a function to handle the group args value change event in text box
def image_args_changed(change):
    new_value = change['new']
    ovner = change['owner']
    image = ovner.image
    image['optionargs'] = new_value

# Create enemuration for group options
class GroupOptions(Enum):
    none = 0
    rename = 1
    del_group_from_list = 2
    merge = 3
    move_to_folder = 4
    deduplicate = 5

    @staticmethod
    def to_list_string():
        optionstrings =[member.name for member in GroupOptions]
        return optionstrings

# Create enemuration for image options
class ImageOptions(Enum):
    none = 0
    del_from_group = 1
    del_from_list = 2
    del_from_hd = 3
    move_to_group = 4

    @staticmethod
    def to_list_string():
        optionstrings =[member.name for member in ImageOptions]
        return optionstrings

print(GroupOptions.to_list_string())

['none', 'rename', 'del_group_from_list', 'merge', 'move_to_folder', 'deduplicate', 'make_items_unique']


In [10]:

def draw_images(data, page=1, images_per_page=10):
    #items_layout = widgets.Layout(align_items='center')
    vertical_frame = widgets.VBox()
    # Calculate the start and end index for the current page
    start_index = (page - 1) * images_per_page
    end_index = start_index + images_per_page
    related_images_per_row = 5

    for group in data[start_index:end_index]:
        group_options = GroupOptions.to_list_string()
        image_options = ImageOptions.to_list_string()

        id = group['id']
        related_images = group['relatedimages']
        group_option_value = group['groupoptions']
        group_option_args = group['optionargs']

        # create row of widgets
        row = widgets.HBox()
        # add horizontal group to vertical group
        baseimagegroup = widgets.VBox()

        # create select checkbox
        select_checkbox = widgets.Checkbox(value=group['selected'], description=str(group['id']))
        select_checkbox.id = id
        select_checkbox.observe(lambda value: set_selected_group(data, value), names='value')
        # create label widget
        label_widget = widgets.Label(value=f'ID: {id} Name: {group["name"]}')
        # create dropdown widget
        dropdown_widget = widgets.Dropdown(options=group_options, value=group_option_value,
                                           description='Group options:')
        dropdown_widget.group = group
        dropdown_widget.observe(group_dropdown_changed, names='value')
        # create group args textbox widget
        group_args_widget = widgets.Text(value=group['optionargs'], description='Group args:')
        group_args_widget.group = group
        group_args_widget.observe(group_args_changed, names='value')

        # add widgets to group
        baseimagegroup.children += (select_checkbox, label_widget, dropdown_widget, group_args_widget)

        row.children += (baseimagegroup,)
        rel_counter = 0
        for relatedimage in related_images:
            id = relatedimage['id']
            # create vertical group
            relatedimagegroup = widgets.VBox()
            # create select checkbox
            select_checkbox = widgets.Checkbox(value=relatedimage['selected'], description=str(id))
            select_checkbox.id = id
            select_checkbox.observe(lambda value: set_selected_group(data, value), names='value')

            # create image widget
            relatedimage_widget = widgets.Image(value=open(relatedimage['relatedimagepath'], 'rb').read(), format='jpg',
                                                width=300,heaight=300)
            # create label widget
            filename = relatedimage['relatedimagepath'].split('\\')[-1]
            label_widget = widgets.Label(
                value=f'ID: {relatedimage["id"]} Name: {filename} Conf: {relatedimage["confidence"]}')

            #create dropdown widget
            dropdown_widget = widgets.Dropdown(options=image_options, value=relatedimage['groupoptions'],description=f'Image options')
            dropdown_widget.image = relatedimage
            dropdown_widget.observe(image_dropdown_changed, names='value')

            # create group args textbox widget
            group_args_widget = widgets.Text(value='', description='Image args:')
            group_args_widget.image = relatedimage
            group_args_widget.observe(image_args_changed, names='value')



            # add widgets to group
            relatedimagegroup.children += (select_checkbox, relatedimage_widget, label_widget, dropdown_widget, group_args_widget)

            if rel_counter % related_images_per_row == related_images_per_row-1:
                # add vertical group to horizontal group
                row.children += (relatedimagegroup,)
                vertical_frame.children += (row,)
                # create new horizontal group
                row = widgets.HBox()
            else:

                # add vertical group to horizontal group
                row.children += (relatedimagegroup,)
            rel_counter += 1

        # Display the related images and labels vertically
        vertical_frame.children+=(row,)
    display(vertical_frame)
    # Create pagination controls
    previous_button = widgets.Button(description='Previous')
    next_button = widgets.Button(description='Next')
    total_pages = (len(data) + images_per_page - 1) // images_per_page
    page_label = widgets.Label(value=f'Page: {page}/{total_pages}')

    # process groups and items button
    process_button = widgets.Button(description='Process actions')

    # Define button click event handlers
    def process_button_clicked(button):

        #rename groups
        for group in data:
            if group['groupoptions'] == GroupOptions.rename.name:
                RenameGroup(group)
        #merge groups
        for group in data:
            if group['groupoptions'] == GroupOptions.merge.name:
                MergeGroup(group)
        #delete groups
        for group in data:
            if group['groupoptions'] == GroupOptions.del_group_from_list.name:
                del_group_from_list(group)

        # move to folder
        for group in data:
            if group['groupoptions'] == GroupOptions.move_to_folder.name:
                move_to_folder(group)

        # deduplicate images in group
        for group in data:
            if group['groupoptions'] == GroupOptions.deduplicate.name:
                deduplicate_images_in_group(group)

        #delete images from group
        for group in data:
            for image in group['relatedimages']:
                if image['groupoptions'] == ImageOptions.del_from_group.name:
                    image_delete_from_group(image)

        # delete images from list
        for group in data:
            for image in group['relatedimages']:
                if image['groupoptions'] == ImageOptions.del_from_list.name:
                    image_delete_from_list(image)


        refresh_page()

    def MergeGroup(group):
        merge_to_group = group['optionargs']
        for igroup in data:
            if igroup['name'] == merge_to_group:
                igroup['relatedimages'] += group['relatedimages']
                break
        #delete group from list
        del_group_from_list(group)

    def del_group_from_list(group):
        data.remove(group)

    def deduplicate_images_in_group(group):
        images = group['relatedimages']
        dict_of_images = {}
        for image in images:
            dict_of_images[image['name']] = image

        group['relatedimages'] = list(dict_of_images.values())

    def move_to_folder(group):
        folder = group['optionargs']

        for image in group['relatedimages']:
            filename = image['relatedimagepath'].split('\\')[-1]
            newpath = os.path.join(folder, filename)
            shutil.move(image['relatedimagepath'], newpath)
            image['relatedimagepath']= newpath


    def RenameGroup(group):
        newname = group['optionargs']
        group['name'] = newname

    # items choises functions
    def image_delete_from_group(image):
        #faind image group
        for group in data:
            if image in group['relatedimages']:
                group['relatedimages'].remove(image)
                break

    def image_delete_from_list(delimage):
        #delete image from list by image path
        for group in data:
            for image in group['relatedimages'].copy():
                if image['relatedimagepath'] == delimage['relatedimagepath']:
                    group['relatedimages'].remove(image)
                    break

    process_button.on_click(process_button_clicked)

    # Define button click event handlers
    def previous_button_clicked(button):
        nonlocal page

        if page > 1:
            page -= 1
            refresh_page()

    def next_button_clicked(button):
        nonlocal page
        total_pages = (len(data) + images_per_page - 1) // images_per_page
        if page < total_pages:
            page += 1
            refresh_page()

    # Refresh the page with updated data
    def refresh_page():
        clear_output()
        draw_images(data, page, images_per_page)

    # Attach event handlers to button clicks
    previous_button.on_click(previous_button_clicked)
    next_button.on_click(next_button_clicked)

    # Display pagination controls
    display(widgets.HBox([previous_button, page_label, next_button, process_button]))


In [53]:
# draw all files
def draw_list_images(images_list,page=1,images_per_page=100):
    vertical_frame = widgets.VBox()

    # Calculate the start and end index for the current page
    start_index = (page - 1) * images_per_page
    end_index = start_index + images_per_page
    related_images_per_row = 5

    for image in images_list:
        id = image['id']
        filepath = image['filepath']
        group_option = image['groupoptions']
        option_args = image['optionargs']
        file_name = filepath.split('\\')[-1]
        # create vertical group
        relatedimagegroup = widgets.VBox()





# gui render

In [54]:
draw_images(listdublicates)

In [43]:
draw_list_images(listdublicates)